In [2]:
from bs4 import BeautifulSoup as bp
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import csv
import time


In [5]:
USER_AGENT='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'
REQUEST_HEADER = {
    'User-Agent': USER_AGENT,
    'Accept-language': 'en-US, en;q=0.5',
}

In [4]:
options=Options()
options.add_argument(f"user-agent={USER_AGENT}")
driver=webdriver.Chrome(options=options)

In [22]:
def get_room_price(soup):
    #prices={}
    #price= soup.find('div',class_='css-901oao r-a5wbuh r-b88u0q r-135wba7 r-1ff274t')
    #prices['base_price']=price
    list=soup.find('div',attrs={'data-testid': 'room-list-tray','class':'css-1dbjc4n r-1oszu61 r-eqz5dr'})
    rooms=list.findAll('div',class_='css-1dbjc4n r-14lw9ot r-1dzdj1l r-xy67v1 r-6gpygo r-1l7z4oj r-ymttw5')
    room_data=[]
    for room in rooms:
        room_info={}
        name=room.find('h3',class_='css-4rbku5 css-901oao css-bfa6kz r-cwxd7f r-a5wbuh r-b88u0q r-135wba7 r-fdjqy7',attrs={'role':'heading'})
        room_info['name']=name.text.strip() if name else 'N/a'
        count=0
        img_box=room.find('div',class_='css-1dbjc4n r-9cy4zw r-h1746q r-1mdj7ya r-15ik8dz r-youum4 r-1l7z4oj r-1e081e0 r-fd4yh7 r-136ojw6')
        dump=img_box.find('svg',attrs={'data-id':'IcUserAccountFill'})
        capa= dump.find_next_sibling('div',class_='css-901oao r-1ud240a r-a5wbuh r-1enofrn r-majxgm r-1cwl3u0 r-fdjqy7')
        if capa:
            div_capa=capa.get_text().strip().replace('x','')
            room_info['capacity']=div_capa    
        else:
            people=img_box.findAll('svg',attrs={'data-id':'IcUserAccountFill'})
            for svg in people:
                count +=1
            room_info['capacity']=count

        price_list = []
        price_elements = room.findAll('div', class_='css-901oao r-1loqt21 r-a5wbuh r-ubezar r-b88u0q r-135wba7 r-1ff274t')
        for price_element in price_elements:
            #take string
            price_text = price_element.get_text().strip()
            #convert to INT
            price_int=int(price_text.replace('VND','').replace('.',''))
            price_list.append(price_int)
            
        room_info['price'] = price_list

        room_data.append(room_info)
    return room_data



In [ ]:
if __name__ =="__main__":
    data=[]
    with open('hotels.csv',newline='',encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        i=0
        for row in reader:
            url=row[0]
            file_name = "room_price.txt"
            driver.get(url)
            time.sleep(15)
            soup=bp(driver.page_source,'lxml')
            data.append(get_room_price(soup))
            with open(file_name, 'w', encoding='utf-8') as file:
                file.write(str(data))